# International Standard Atmosphere
U.S. Standard Atmosphere - 1976

Version control
* v1.0 initial release

Starting with the Equation of State for ideal gases:

$PV=nRT$

$P=\rho RT$

Where R is the specific gas constant, which for air is $R=287.05$ $\frac{J}{kgK}$ in the SI.

<img src="pictures/diff_atmos_element.png" alt="drawing" width="300"/>

From the hydrostatic equation:

$W=-mg$

$W=-(\rho * V)g$

$W=-\rho * (dA * dh) g$

Note that $h$ is the tapeline height.

Force due to pressure:

$\sum F_p=PdA-(P+dP)dA$

$\sum F_p=-dPdA$

All forces:

$\sum F=-\rho dAdhg-dPdA=0$

or:

$dP=-\rho g dh$

We could try:

$\int_{0}^{P}{dP}=\int_{0}^{h}(-\rho g) dh$

But since *g* varies with altitude, and so does density ... we would have to do a numerical integral...

So, we have a slight problem. 

But how does gravity vary with altitude...

$g=g_{SL}(\frac{R_{eSL}}{R_{eSL}+h})^2$  -> per Newton's theory

But, how much is this for the altitudes we usually fly?

In [ ]:
ft2m = 1 / 3.28084
Radius_Earth = 6378000 # [m]
g_SL = 9.8065 # [m/s2]
my_altitude = 55000 # [ft]
g_my_alt = g_SL*(Radius_Earth/(Radius_Earth + (my_altitude * ft2m)))**2
print(f'accel of gravity at {my_altitude} ft is {g_my_alt} m/s2, with a delta of {g_SL - g_my_alt} m/s2')


Let's define *geopotential altitude (H)* as

$gdh=g_{SL}dH$ ...

Substitute and integrate:

$dH=(\frac{R_{eSL}}{R_{eSL}+h})^2dh$

$\int_{0}^{H}dH=\int_{0}^{h}(\frac{R_{eSL}}{R_{eSL}+h})^2dh$

Let’s learn a cool trick to do integrals (and other calculus) in Python: 

## Sympy

In [ ]:
# https://docs.sympy.org/latest/index.html
# tut https://www.youtube.com/watch?v=1yBPEPhq54M
import sympy as smp

### Basics

symbolic variables

In [ ]:
x = smp.symbols('x')
x

multiple variables, latex

In [ ]:
k1, z = smp.symbols('k_1 z')

In [ ]:
k1

In [ ]:
# simple math operations
x**2

In [ ]:
smp.sqrt(x)

In [ ]:
# capturing the results in a new variable
y = x**2
y

In [ ]:
# polynomials

y = x**2 - 8*x + 12
y

In [ ]:
# first argument (y) is equal to zero - trick with equations, just move everything to one side
smp.solve(y, x)

In [ ]:
y = y**2
y

In [ ]:
y.factor()

In [ ]:
y.expand()

In [ ]:
# assumptions
x = smp.symbols('x')
smp.solve(x**2-1*x+2,x)

In [ ]:
x = smp.symbols('x', real=True)
smp.solve(x**2-1*x+2,x)

In [ ]:
# multivariables
x, y, z = smp.symbols('x y z')
F = x**2 + smp.sin(z) * y
F

In [ ]:
smp.solve(F,x)

In [ ]:
smp.solve(F,z)

In [ ]:
expr = smp.solve(F, z)[0]
expr

In [ ]:
expr_f = smp.lambdify([x, y], expr)
expr_f(1,2)

In [ ]:
# derivative

smp.diff(F, z)

In [ ]:
# integral

smp.integrate(F, z)

In [ ]:
# tab
smp.

In [ ]:
# shift - tab
smp.exp

In [ ]:
R_eSL, h, H = smp.symbols('R_{eSL}, h, H', real=True, positive=True)

In [ ]:
dH = (R_eSL / (R_eSL + h))**2
dH

In [ ]:
H = smp.integrate(dH, (h,0,h))
H

In [ ]:
H = smp.integrate(dH, (h,0,h)).simplify()
H

In [ ]:
H_f = smp.lambdify([R_eSL, h], H)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background')
%matplotlib widget

In [ ]:
# conversion factors
m2ft = 3.28084
ft2m = 1 / m2ft
kt2ms = 1852 / 60 / 60 # 1 nm in meters; from hours to seconds
ms2kt = 1 / kt2ms
C2K = 273.15
d2r = np.pi / 180
r2d = 1 / d2r

In [ ]:
h = np.arange(0,20000,1000) # [m]

In [ ]:
g_SL = 9.80665 # [m/s2]
#WGS84 geoid
a = 6378137.0 # [m] - semi-major axis
b = 6356752.3142 # [m] - semi-major axis
latitude = 45 # [deg]
R_eSL = np.sqrt(((a**2*np.cos(latitude*d2r))**2+(b**2*np.sin(latitude*d2r))**2)/
                ((a*np.cos(latitude))**2+(b*np.sin(latitude))**2)) # [m]
#H = R_eSL * h / (R_eSL + h) # [m]
H_res = H_f(R_eSL, h) # [m]

In [ ]:
fig,ax = plt.subplots(1)

ax.plot(H_res*m2ft,(h-H_res)*m2ft) # [ft] vs [ft]
ax.format_coord = lambda x, y: f"H={x:5.0f}, h-H={y:3.1f}"
ax.set_xlabel('H [ft]')
ax.set_ylabel('h-H [ft]')

ax.grid(visible=True, alpha=0.2)
plt.show()

As we can see, if we switch from geometric altitude to geopotential altitude, the impact is not significative up to 50,000ft (~120ft).



________________________________________________

So we can rewrite the hydrostatic equation with geopotential height instead of tapeline height:

$dP=-\rho g_{SL} dH$

Coming back to our model of the atmosphere, now we can divide the Hydrostatic Equation by the Equation of State to get:

$\frac{dP}{P}=\frac{-\rho g_{SL}dH}{\rho RT}$

$\frac{dP}{P}=\frac{- g_{SL}dH}{ RT}$

## Troposphere

For the region where temperature varies, we define a **constant lapse rate**:

$L=\frac{dT}{dH}$

We rearrange and integrate:

$LdH=dT$

$L \int_{H_{bottom}}^{H}dH=\int_{T_{bottom}}^{T}dT$

$L (H-H_{bottom}) =T-T_{bottom}$

Isolate temperature:

$T=T_{bottom}+L (H-H_{bottom})$

Divide by $T_{bottom}$ to get the ratio:

$\frac{T}{T_{bottom}}=1+\frac{L}{T_{bottom}} (H-H_{bottom})$

We had: $\frac{dP}{P}=\frac{- g_{SL}dH}{ RT}$

$L=\frac{dT}{dH}$ --> isolate $dH=\frac{dT}{L}$

$\frac{dP}{P}=\frac{- g_{SL}}{LR}\frac{dT}{T}$

Let's use Sympy here again:

In [ ]:
P, P_bottom = smp.symbols('P, P_{bottom}', real=True, positive=True)
H, H_bottom = smp.symbols('H, H_{bottom}', real=True, positive=True)
T, T_bottom = smp.symbols('T, T_{bottom}', real=True, positive=True)
g_SL = smp.symbols('g_{SL}', real=True, positive=True)
R, L = smp.symbols('R, L', real=True, positive=True)

In [ ]:
lhs = smp.integrate(1/P, (P,P_bottom,P)).simplify()
lhs

In [ ]:
rhs = -g_SL/(R*L)*smp.integrate((1)/T, (T,T_bottom,T)).simplify()
rhs

we can now raise both sides to $e$:

In [ ]:
smp.exp(lhs)

In [ ]:
smp.exp(rhs).simplify()

Notice that although Simpy gives us $\frac{T_{bottom}}{T}$, we can invert this by taking the negative expoent

$\frac{P}{P_{bottom}}=\frac{T}{T_{bottom}}^{\frac{- g_{SL}}{LR}}$

substituting:
$\frac{T}{T_{bottom}}=1+\frac{L}{T_{bottom}} (H-H_{bottom})$

$\frac{P}{P_{bottom}}=(1+\frac{L}{T_{bottom}} (H-H_{bottom}))^{\frac{- g_{SL}}{LR}}$

___________________

If now we define values for our standard atmosphere, at mean seal level:

* $P_{bottom}=P_{SL}$
* $H_{bottom}=0$
* $T_{bottom}=T_{SL}$

______________________________

and define the pressure ratio 

$\delta=\frac{P}{P_{SL}}$

We can write:

$\delta=(1+\frac{L}{T_{SL}} (H-0))^{\frac{- g_{SL}}{LR}}$

$\delta=(1+\frac{L}{T_{SL}} (H))^{\frac{- g_{SL}}{LR}}$

Finally, we DEFINE "pressure altitude" as the altitude obtained from the standard atmosphere equation, for a given pressure. This allows us to replace the geopotential height ($H$) by pressure altitude ($H_c$)

$\delta=(1+\frac{L}{T_{SL}} (H_c))^{\frac{- g_{SL}}{LR}}$

We can also obtain the equations for density, by using the equation of state:

$\frac{P}{P_{bottom}}=\frac{\rho R T}{\rho_{bottom}R T_{bottom}}$

$\frac{\rho}{\rho_{bottom}}=\frac{P}{P_{bottom}}*(\frac{T}{T_{bottom}})^{-1}$

Fetching what we had before:

$\frac{P}{P_{bottom}}=\frac{T}{T_{bottom}}^{\frac{- g_{SL}}{LR}}$

so now we can substitute:

$\frac{\rho}{\rho_{bottom}}=\frac{T}{T_{bottom}}^{\frac{- g_{SL}}{LR}}*(\frac{T}{T_{bottom}})^{-1}$

$\frac{\rho}{\rho_{bottom}}=\frac{T}{T_{bottom}}^{\frac{- g_{SL}}{LR}-1}$

And since we had also

$\frac{T}{T_{bottom}}=1+\frac{L}{T_{bottom}} (H-H_{bottom})$

we can substitute as well to get:

$\frac{\rho}{\rho_{bottom}}=(1+\frac{L}{T_{bottom}} (H-H_{bottom}))^{\frac{- g_{SL}}{LR}-1}$

Define density ratio:

$\sigma=\frac{\rho}{\rho_{SL}}$

For standard atmosphere, we can write:


$\sigma=(1+\frac{L}{T_{SL}} (H_c))^{\frac{- g_{SL}}{LR}-1}$

The last ratio that we define is for temperature:

$\theta=\frac{T}{T_{SL}}$

We had:

$\frac{T}{T_{bottom}}=1+\frac{L}{T_{bottom}} (H-H_{bottom})$

Which, for the standard atmosphere, becomes:

$\theta=1+\frac{L}{T_{SL}} (H_c)$

________________________________

## Stratosphere
If temperature is **constant**, we can easily integrate:

$\int_{P_{bottom}}^{P}\frac{dP}{P}=-\frac{g_{SL}}{RT_{bottom}}\int_{H_{bottom}}^{H}dH$

$ln(P)-ln(P_{bottom})=-\frac{g_{SL}}{RT_{bottom}}(H-H_{bottom})$

$ln(\frac{P}{P_{bottom}})=-\frac{g_{SL}}{RT_{bottom}}(H-H_{bottom})$

$\frac{P}{P_{bottom}}=e^{-\frac{g_{SL}}{RT_{bottom}}(H-H_{bottom})}$

Now, $\delta=\frac{P}{P_{SL}}=\frac{P_{bottom}}{P_{SL}}\frac{P}{P_{bottom}}=\delta_{bottom}\frac{P}{P_{bottom}}$

$\delta=\delta_{bottom}\frac{P}{P_{bottom}}$

So, in this case, $\frac{P}{P_{bottom}}=\frac{\delta}{\delta_{bottom}}$

and write for the standard atmosphere, with the subscript "strato-bot" meaning the value at the bottom of the stratosphere:

$\delta=\delta_{strato-bot}(e^{-\frac{g_{SL}}{RT_{strato-bot}}(H_c-H_{strato-bot})})$


Similarly, for the density equation:

$\frac{P}{P_{bottom}}=\frac{\rho R T}{\rho_{bottom}R T_{bottom}}$

Since $T=T_{bottom}$= constant

$\frac{P}{P_{bottom}}=\frac{\rho}{\rho_{bottom}}$

$\frac{\rho}{\rho_{bottom}}=e^{-\frac{g_{SL}}{RT_{bottom}}(H-H_{bottm})}$

$\sigma=\sigma_{strato-bot}(e^{-\frac{g_{SL}}{RT_{strato-bot}}(H_c-H_{strato-bot})})$

Last, it is useful to have the inverse relation for pressure altitude: given a pressure, we want to know the pressure altitude. 

These can be obtained by manipulating the equations we already have and will just be listed below.

Standard Atmosphere Summary:

| Direct Relations                | Troposphere                                             | Stratosphere                                                           |
| :--                             | :-:                                                     | :-:                                                                    |
| $\delta=\frac{P}{P_{SL}}$       | $\delta=(1+\frac{L}{T_{SL}}(H_c))^{\frac{-g_{SL}}{LR}}$ | $\delta=\delta_{strato-bot}(e^{-\frac{g_{SL}}{RT_{strato-bot}}(H_c-H_{strato-bot})})$ |
| $\sigma=\frac{\rho}{\rho_{SL}}$ | $\sigma=(1+\frac{L}{T_{SL}}(H_c))^{\frac{-g_{SL}}{LR}-1}$ | $\sigma=\sigma_{strato-bot}(e^{-\frac{g_{SL}}{RT_{strato-bot}}(H_c-H_{strato-bot})})$ |
| $\theta=\frac{T}{T_{SL}}$       | $\theta=1+\frac{L}{T_{SL}}(H_c)$                        | $\theta=\frac{T_{strato-bot}}{T_{SL}}$                                  |
| __________________ | ____________________________ | _____________________________________________________________ |
| **Inverse Relations**               |                                                         |                                                                             |
| given pressure       | $H_c=\frac{T_{SL}}{L}(\delta^{-\frac{LR}{g_{SL}}}-1)$ | $H_c=\frac{-RT_{strato-bot}}{g_{SL}}log(\frac{\delta^{(\frac{-g_{SL}}{RT_{strato-bot}}{H_{strato-bot}})}}{\delta_{strato-bot}})$ |
| given density | $H_{\rho}=\frac{T_{SL}}{L}(\sigma^{\frac{1}{\frac{-g_{SL}}{LR}-1}}-1)$ | $H_{\rho}=\frac{-RT_{strato-bot}}{g_{SL}}log(\frac{\sigma^{(\frac{-g_{SL}}{LT_{strato-bot}}{H_{strato-bot}})}}{\sigma_{strato-bot}})$ |
| __________________ | ____________________________ | _____________________________________________________________ |
| Altitude  | $H_{tropo-bot} = 0m$  | $H_{strato-bot}=11000m = ft =36089.24$   |
| Temperature  | $T_{tropo-bot} = 288.15K$  | $T_{strato-bot}=216.65K$  |
| Pressure  | $P_{tropo-bot} = 101325Pa$  | $P_{strato-bot}=22632.06Pa$  |
| Density  | $\rho_{tropo-bot} = 1.225 \frac{kg}{m^3}$  | $\rho_{strato-bot}=0.363918 \frac{kg}{m^3}$  |
| __________________ | ____________________________ | _____________________________________________________________ |
